# Flexible Single Body Models (no rigid displacements)

The goal of this project is to consider flexible single body models. We assume the door to be a flexible bar (or plate) attached to the bus. We wish to take the structural properties of the door (deformation, strain and stress) into account. We assume (for simplicity) that the door is a linear (small displacement around point of equilibrium in which the door is closed) and homogeneous (constant Lame parameters or Youngs' modulus) elastic medium. We wish to compare e.g. a rigid door and flexible suspension (expect large rigid displacement of the door, recover previous cases) with a flexible door and stiff suspension (expect large deformation of the door, recover case of flexible bar attached to reference). 

As before, we aim at 
1. a static analysis: one is typically interested in maximal displacements and stresses. For these quantities, analytical expressions do exist for the bar [using cantilever beam example for Euler–Bernoulli beam theory](https://en.wikipedia.org/wiki/Euler–Bernoulli_beam_theory) and for the plate using the [rectangular Kirchhoff-Love plate theory](https://en.wikipedia.org/wiki/Bending_of_plates). A comparison between analytical and numerical computations in made in [this example of the GXBEAM.jl package](https://flow.byu.edu/GXBeam.jl/dev/examples/tipforce/); 
2. a transient ansalysis: In a transient analysis, some form of the wave-equation needs to be solved (vibrasting plate theory). This can be handled by DifferentialEquations.jl. 
3. an eigenmode analysis; 
4. a vibration analysis; 

## Import Packages

In [12]:
using LinearAlgebra
using DifferentialEquations
using SparseArrays
using Plots
using LaTeXStrings
using BenchmarkTools 

## Section 1:/ Introduction 

### Theory 
We will use the finite element method for (non-)linear elastic media. For theory, geometry definition, mesh generation and references see [EE4375 Course](https://github.com/ziolai/finite_element_electrical_engineering). 

### Implementation 

#### Using Gridap 

#### Alternative Tools to use 
1. [FinEtoolsFlexBeams](https://github.com/PetrKryslUCSD/FinEtoolsFlexBeamsTutorials.jl): includes vibrational analysis and assembly of beams;  
2. [GXBEAM](https://flow.byu.edu/GXBeam.jl/dev/): implementation of Geometrically Exact Beam Theory; includes vibrational analysis and assembly of beams; 
3. [the finite element package GridAp.jl](https://github.com/gridap/Gridap.jl) allows to describe the elastic deformation of flexible bodies.

## Section 2:/ Flexible Bar Models 

In [1]:
# this example was provided by Oriol Colomes on the #gridal Slack channel on July 31st 
#module EB_beam # Euler-Bernouilli 
using Gridap
# model = CartesianDiscreteModel((0,1),(10))  # Domain from 0 to 1, discretized with 10 elements
model = CartesianDiscreteModel((0,1),(100))  # Domain from 0 to 1, discretized with 10 elements
Ω = Interior(model) # Triangulation
Λ = Skeleton(Ω)     # Triangulation of the skeleton (interior nodes) -> used in the CG/DG approach to enforce continuity of rotations weakly
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left",[1,])  # Tag the left end node
add_tag_from_tags!(labels,"right",[2,]) # Tag the right end node (assume Free Neumann BC)
ΓD = Boundary(Ω,tags="left")
order = 1 # order = 2 # FE polynomial degree
refFE = ReferenceFE(lagrangian,Float64,order) # Lagrangian reference FE
V = TestFESpace(Ω,refFE,dirichlet_tags="left") # FE Space with Dirichlet BC at the left node
U = TrialFESpace(V,0.0) # Trial FE space with 0.0 deflection value at the Dirichlet BC
EI = 5.0e5 # Rigidity
h = 1/100   # Element size
γ = 1.0    # penalty constant
dΩ = Measure(Ω,2*order) # Integration measure for Ω
dΓ = Measure(ΓD,2*order) # Integration measure for ΓD
dΛ = Measure(Λ,2*order) # Integration measure for Λ
nΓ = get_normal_vector(ΓD) # normal vector at the boundary (this is a 1D vector in that case)
nΛ = get_normal_vector(Λ) # normal vector at the skeleton (this is a 1D vector in that case)
a(u,v) = ∫( EI*(Δ(u)*Δ(v)) )dΩ +
         ∫( EI * ( - Δ(u)*(∇(v)⋅nΓ) - (∇(u)⋅nΓ)*Δ(v) + γ/h*(∇(u)⋅nΓ)*(∇(v)⋅nΓ) ) )dΓ +
         ∫( EI * ( - mean(Δ(u))*jump(∇(v)⋅nΛ) - jump(∇(u)⋅nΛ)*mean(Δ(v)) + γ/h*jump(∇(u)⋅nΛ)*jump(∇(v)⋅nΛ) ) )dΛ
l(v) = ∫( -1.0*v )dΩ  # negative distributed force
op = AffineFEOperator(a,l,U,V)  # FE operator
uh = solve(op)
writevtk(Ω,"solution",cellfields=["u"=>uh])
#end

(["solution.vtu"],)

In [2]:
uhdof = get_free_dof_values(uh)

100-element Vector{Float64}:
 -1.0000000000051267e-10
 -2.9801000000153544e-10
 -5.920600000030658e-10
 -9.802000000051006e-10
 -1.4605000000076368e-9
 -2.0310500000106714e-9
 -2.6899600000142015e-9
 -3.435360000018224e-9
 -4.2654000000227365e-9
 -5.178250000027736e-9
 -6.172100000033217e-9
 -7.2451600000391805e-9
 -8.395660000045621e-9
  ⋮
 -2.178186000017063e-7
 -2.2119825000174063e-7
 -2.2457890000177517e-7
 -2.2796036000180994e-7
 -2.3134246000184495e-7
 -2.347250500018802e-7
 -2.3810800000191565e-7
 -2.4149120000195134e-7
 -2.4487456000198726e-7
 -2.482580100020234e-7
 -2.516415000020597e-7
 -2.55025000002096e-7

In [3]:
plot(uhdof,label="Displacement", fmt = :png)
xlabel!("x") 
ylabel!("displacement")

LoadError: UndefVarError: plot not defined

## Section 3:/ Flexible Plate Models 
Using (non-)linear elasticity. 

- Linear elasticity tutorial in [GridAp](https://github.com/gridap/Gridap.jl): [tutorial-3](https://gridap.github.io/Tutorials/stable/pages/t003_elasticity/#Tutorial-3:-Linear-elasticity-1) 

- Non-Linear Elasticity Tutorial in [GridAp](https://github.com/gridap/Gridap.jl): [tutorial-5](https://gridap.github.io/Tutorials/stable/pages/t005_hyperelasticity/)

- Models previously developed by XiEngineering and how to extend them. 

## References 